# MACS30122 Final Project - Record Linkage

## Background Infomation
### Data sources

For this project, we have 4 main data sources in 55 separate files in total.

**Twitter Data**
- CDC tweets → "data/cdc_twitter_covid.json"
- State governors' tweets → "data/governor", 52 separate JSON files named by the state

**Covid Data**
- Daily number of cases by country → "data/covid-data-US.csv"
- Daily number of cases by state → "data/covid-data-by-state.csv"

### Sample output of the record linkage

Our goal is to generate a "big table" to serve data for the dashboard.  
Here is the sample logic for the "big table". The primary key will be the date, and the table is further partitioned by state.

<img style="float: left;" src="Sample_output_rl.png"> 

Before getting to the code, let's input some magic functions, packages, and data paths.  
The helper functions of the record linkage process are stored in a file named Record_linkage_helper.py.  
Here we also use two clean-up helper functions from covid_data_analysis.py.

In [38]:
# Magic functions
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [39]:
# Packages and paths

import pandas as pd
import os

import Record_linkage_helper as RL
from covid_data_analysis import get_correct_date, clean_data

p='data/governor/'
p_covid_data = 'data/'

## Step 1: Generate a "big table" of the daily number of cases by country and state

The first step of the record linkage is to merge covid data by country and by state.  
We read in the daily number of cases by country and state from the CSV files, and preprocess them into readable Dataframes. Then, we merge the two sources of data to get a big table of covid data.

In [40]:
# Daily number of cases by country
covid_data_US = get_correct_date(pd.read_csv(os.path.join(p_covid_data, "covid-data-US.csv"),encoding='utf-8'))

# Daily number of cases by state
covid_data_state = clean_data(pd.read_csv(os.path.join(p_covid_data, "covid-data-by-state.csv"),encoding='utf-8'))
covid_data_state.loc[:, 'date'] = covid_data_state.loc[:, 'date'].apply(RL.replace_separator)
covid_data_state = covid_data_state[['province_state','date','confirmed_state','deaths_state']]

# Merge two sources of covid data
merged_covid_data = pd.merge(left = covid_data_state,right = covid_data_US, on = 'date',how = 'outer')
merged_covid_data.sort_values(['date','province_state'],inplace = True)
merged_covid_data.reset_index(drop=True, inplace=True)

Let's take a look at what the initial big table looks like. Note that here we use an outer join to cover all available data either by country or by state.

In [41]:
merged_covid_data

,province_state,date,confirmed_state,deaths_state,confirmed,deaths
0,NaN,2020/01/22,NaN,NaN,1,NaN
1,NaN,2020/01/23,NaN,NaN,1,NaN
2,NaN,2020/01/24,NaN,NaN,2,NaN
3,NaN,2020/01/25,NaN,NaN,2,NaN
4,NaN,2020/01/26,NaN,NaN,5,NaN
...,...,...,...,...,...,...
18877,Virginia,2021/02/21,564115.0,7331.0,28134115,498897.0
18878,Washington,2021/02/21,333794.0,4822.0,28134115,498897.0
18879,West Virginia,2021/02/21,129616.0,2261.0,28134115,498897.0
18880,Wisconsin,2021/02/21,612240.0,6871.0,28134115,498897.0


## Step 2: Merge the CDC tweets data

In [42]:
# Read in and clean CDC tweets

cdc_tweets_df = pd.read_json(os.path.join(p_covid_data, 'cdc_twitter_covid.json'),lines=True,encoding='utf-8')

cdc_tweets_df.loc[:, 'date'] = cdc_tweets_df.loc[:, 'date'].apply(RL.timestamp_to_str)
cdc_tweets_df = cdc_tweets_df[['tweet','date']]
cdc_tweets_df['tweet'] = cdc_tweets_df['tweet'].apply(RL.remove_u2069)
cdc_tweets_df.rename(columns={'tweet': 'CDC_tweet'}, inplace=True)

We read in and clean the CDC tweets. Take a look at the data. And link it to the big table.

In [43]:
cdc_tweets_df.head()

,CDC_tweet,date
0,#DYK? COVID-19 and flu can both cause fever an...,2021/02/22
1,A new @CDCMMWR looks at #COVID19 in Georgia el...,2021/02/22
2,Officials are concerned about 3 #COVID19 varia...,2021/02/19
3,Getting a #COVID19 vaccine is important for ad...,2021/02/19
4,#DYK? You need two #COVID19 mRNA vaccine doses...,2021/02/18


In [44]:
merged_covid_cdc_tweet_df = pd.merge(left = merged_covid_data,right = cdc_tweets_df, on = 'date',how = 'left')

## Step 3: Merge the governors' tweets

To merge tweets from each governor, we create a function to read in and clean data from one JSON file, and then iterate the function on all JSON files we have.  
First, let's see how the helper function works on one single JSON file.

In [45]:
RL.read_in_one_governor_tweets(p,'Alabama.json')

,governor_tweet,date,province_state
0,Great news! 🚨Alabama ranks 5th in Back-to-Norm...,2021/03/09,Alabama
1,A great corporate partner for Alabama during #...,2020/08/25,Alabama
2,Absolutely! And they will - I appreciate you f...,2020/05/15,Alabama
3,There are several #COVID19 testing centers ava...,2020/05/14,Alabama
4,ICYMI: @ALWorksSystem has released the AL COVI...,2020/04/15,Alabama
5,My priority as governor is making sure every A...,2020/04/09,Alabama
6,It is with profound sadness that I confirm tha...,2020/03/25,Alabama


Starting from there, we store all file names in a list called all_json_state_names, and apply the helper function on all JSON files.

In [46]:
# Read in tweets from all governors

all_governor_tweets_lst = []
for i in RL.all_json_state_names:
    file_name = i+'.json'
    print("Finish reading in tweets from the governor of "+i)
    t = RL.read_in_one_governor_tweets(p,file_name)
    all_governor_tweets_lst.append(t)

# Concat
all_governor_tweets = pd.concat(all_governor_tweets_lst)
all_governor_tweets.sort_values(['date','province_state'],inplace = True)
all_governor_tweets.reset_index(drop=True, inplace=True)

print("Finish reading in tweets from "+str(len(all_governor_tweets_lst))+" governors, getting "+str(len(all_governor_tweets))+" tweets")

Finish reading in tweets from the governor of Alaska
Finish reading in tweets from the governor of Alabama
Finish reading in tweets from the governor of Arkansas
Finish reading in tweets from the governor of Arizona
Finish reading in tweets from the governor of California
Finish reading in tweets from the governor of Colorado
Finish reading in tweets from the governor of Connecticut
Finish reading in tweets from the governor of DC
Finish reading in tweets from the governor of Delaware
Finish reading in tweets from the governor of Florida
Finish reading in tweets from the governor of Georgia
Finish reading in tweets from the governor of Hawaii
Finish reading in tweets from the governor of Iowa
Finish reading in tweets from the governor of Idaho
Finish reading in tweets from the governor of Illinois
Finish reading in tweets from the governor of Indiana
Finish reading in tweets from the governor of Kansas
Finish reading in tweets from the governor of Kentucky
Finish reading in tweets from

Remember the tweets from the governor of Alabama above, let's double-check them in the dataframe of all governors' tweets.

In [47]:
all_governor_tweets[all_governor_tweets['province_state']=='Alabama']

,governor_tweet,date,province_state
1036,It is with profound sadness that I confirm tha...,2020/03/25,Alabama
1856,My priority as governor is making sure every A...,2020/04/09,Alabama
2132,ICYMI: @ALWorksSystem has released the AL COVI...,2020/04/15,Alabama
3346,There are several #COVID19 testing centers ava...,2020/05/14,Alabama
3395,Absolutely! And they will - I appreciate you f...,2020/05/15,Alabama
6140,A great corporate partner for Alabama during #...,2020/08/25,Alabama
11859,Great news! 🚨Alabama ranks 5th in Back-to-Norm...,2021/03/09,Alabama


## Step 4: Final merge

At this point, we have finished record linkage from almost all data sources. Let's generate a final dataframe and output it into a CSV file.

In [48]:
merged_df_final = pd.merge(left = merged_covid_cdc_tweet_df,right = all_governor_tweets, \
                                 on = ['date','province_state'],how = 'left')
merged_df_final.to_csv(os.path.join(p_covid_data,"merged_df_final.csv"))

Let's try to print one of the tweets sent by the governor of Hawaii on Mar 17, 2020. We have kept the emojis, URLs and hashtags within the tweets to ensure that the tweets are still readable in the dashboard.

In [49]:
test_tweet = pd.read_csv(os.path.join(p_covid_data,"merged_df_final.csv"))
test_tweet.governor_tweet[733]

'#Live News Conference: COVID-19 re: 15 days to slow spread of #coronavirus ➡️ https://t.co/C5YJFYrPG0  https://t.co/srA9ghNMTd'

Now we get the data ready. Let's move on to the the visualization and see how the data look like in the dashboard.